# CrewAI C.V. Agents

## The Goals

1. Analyze the job posting to extract structured requirements.
2. Profile a candidate by querying a knowledge base for data relevant to the job posting
3. Optimize the candidate's CV based on the job requirements and profile data

## The Data Flow

1. Job posting URL → Job Analyst → Structured job requirements
2. Candidate data + Job requirements → Candidate Profiler → Profile analysis  
3. Profile + Job requirements → CV Strategist → Optimization recommendations
4. Optimized data → Template engine → LaTeX → PDF

### Basic Setup

*For displaying source files*

In [ ]:
from IPython.display import Markdown

*The Job Posting and Output Directory*

In [ ]:
job_posting_url = "https://app.welcometothejungle.com/dashboard/jobs/oA1SArxV"
output_directory = "output/automattic"

### The Job Analyst

In [ ]:
with open('optimizer/agents/job_analyst.py') as file:
    content = file.read()
display(Markdown('```\n' + content + '\n```'))

In [ ]:
from optimizer.agents.job_analyst import job_analyst as analyst

### The Candidate Profiler

In [ ]:
with open('optimizer/agents/candidate_profiler.py') as file:
    content = file.read()
display(Markdown('```\n' + content + '\n```'))

In [ ]:
from optimizer.agents.candidate_profiler import candidate_profiler as profiler

### The C.V. Strategist

In [ ]:
with open('optimizer/agents/job_analyst.py') as file:
    content = file.read()
display(Markdown('```\n' + content + '\n```'))

In [ ]:
from optimizer.agents.cv_strategist import cv_strategist as strategist

### The Job Analysis Task

In [ ]:
with open('optimizer/tasks/job_analysis_task.py') as file:
    content = file.read()
display(Markdown('```\n' + content + '\n```'))

In [ ]:
from optimizer.tasks.job_analysis_task import create_job_analysis_task


### The Candidate Profiling Task

In [ ]:
with open('optimizer/tasks/candidate_profiling_task.py') as file:
    content = file.read()
display(Markdown('```\n' + content + '\n```'))

In [ ]:
from optimizer.tasks.candidate_profiling_task import create_candidate_profiling_task


### The CV Optimization Task

In [ ]:
with open('optimizer/tasks/cv_optimization_task.py') as file:
    content = file.read()
display(Markdown('```\n' + content + '\n```'))

In [ ]:
from optimizer.tasks.cv_optimization_task import create_cv_optimization_task


*Each task is coupled to the previous task in the pipeline*

In [ ]:
job_analysis = create_job_analysis_task(analyst)
candidate_profiling = create_candidate_profiling_task(profiler, job_analysis)
cv_optimization = create_cv_optimization_task(strategist, job_analysis, candidate_profiling)

### The Crew

In [ ]:
from crewai import Crew, LLM
from config import get_config

config = get_config()

inputs = {
    "job_posting_url": job_posting_url,
    "cv_data_path": "data/cv.yaml",
    "output_directory": output_directory,
}

llm = LLM(
    model=config.crew_manager_model, temperature=float(config.crew_manager_temperature),
)

crew = Crew(
    agents=[analyst, profiler, strategist],
    tasks=[job_analysis, candidate_profiling, cv_optimization],
    verbose=True,
)

*Release the hounds!*

In [ ]:
result = crew.kickoff(inputs=inputs)